# Real estate agent chatbot

A generative AI chatbot that acts as a friendly and knowledgeable real estate agent. It uses OpenAI's models and vector search to retrieve relevant listings and answer user questions.


In [1]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 9.5 MB/s eta 0:00:0

In [2]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from typing import List

# Load OpenAI API key
API_KEY = 'YOUR OPENAI KEY'
os.environ['OPENAI_API_KEY'] = API_KEY
api_key = os.getenv('OPENAI_API_KEY')

## Generate the real estate listings

In [3]:
def generate_listings(model_name: str, temperature: float, amount: int, path: str = "listings.txt") -> None:
    """Generates property listings using an LLM and saves them to a file.

    Args:
        model_name (str): The name of the LLM model to use.
        temperature (float): The temperature parameter for the LLM.
        amount (int): The number of listings to generate. This parameter is used to modify the prompt to request the desired number of listings.
        path (str, optional): The path to the file where listings will be saved. Defaults to "listings.txt".

    Raises:
        Exception: If there is an error during the LLM interaction or file writing.
    """
    # Create the prompt to generate the listings
    prompt = f"""You're helping to build a generative AI chatbot that acts as a friendly and knowledgeable real estate agent. Please generate {amount} realistic and diverse real estate property listings. Format each listing exactly as shown below, using the same field names and order for consistency. Use clear separators between listings.
    Format:
    === Listing ===
    Property Title: [Title]
    Location: [City, Neighborhood]
    Price: $[Price]
    Bedrooms: [#]
    Bathrooms: [#]
    Square Footage: [#] sq ft
    Key Features: [List key features like garage, pool, renovated kitchen, etc.]
    Description: [2–3 sentence summary from an agent’s perspective]
    """
    try:
        # Initialize the LLM
        llm = ChatOpenAI(model=model_name, temperature=temperature)

        # Generate the listings
        response = llm([HumanMessage(content=prompt)])

        # Save the listings to a file
        with open(path, "w", encoding="utf-8") as f:
            f.write(response.content)

    except Exception as e:
        raise Exception(f"An error occurred: {e}")

In [ ]:
# Generate and save the property listings
model_name = 'gpt-4o-mini'
temperature=0.5
amount = 10
generate_listings(model_name=model_name, temperature=temperature, amount=amount)

## Load the listings

In [5]:
def load_listings(path: str) -> str:
    """Loads property listings from a text file.

    Args:
        path (str): The path to the text file containing the listings.

    Returns:
        str: The content of the listings file as a single string.

    Raises:
        FileNotFoundError: If the file specified by 'path' is not found.
    """
    try:
        with open(path, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        raise FileNotFoundError(f"Listings file not found at path: {path}")

In [6]:
full_text = load_listings('listings.txt')

## Store listings in a vector database

In [7]:
listings = [listing.strip() for listing in full_text.split("=== Listing ===") if listing.strip()]
print(listings)

['Property Title: Charming Bungalow in Historic District  \nLocation: Charleston, Ansonborough  \nPrice: $525,000  \nBedrooms: 3  \nBathrooms: 2  \nSquare Footage: 1,500 sq ft  \nKey Features: Original hardwood floors, large backyard, updated kitchen, walking distance to downtown  \nDescription: This delightful bungalow offers a perfect blend of historic charm and modern convenience. With its spacious backyard and proximity to local shops and restaurants, it’s an ideal home for families and professionals alike.\n\n---', 'Property Title: Modern Condo with City Views  \nLocation: New York, Lower East Side  \nPrice: $1,200,000  \nBedrooms: 2  \nBathrooms: 2  \nSquare Footage: 1,100 sq ft  \nKey Features: Rooftop terrace, gym, stainless steel appliances, open floor plan  \nDescription: Experience urban living at its finest in this stylish condo featuring breathtaking city views. The open layout and luxurious amenities make it a perfect retreat in the heart of the vibrant Lower East Side.\n

In [8]:
def create_vector_store(listings_text: List[str], embedding_model: OpenAIEmbeddings) -> Chroma:
    """Creates a vector store from property listings using ChromaDB.

    This function splits the listings into smaller chunks using a RecursiveCharacterTextSplitter,
    embeds them using the provided embedding model, and stores them in a ChromaDB vector store.

    Args:
        listings_text: A list of property listing strings.
        embedding_model: The OpenAIEmbeddings model to use for generating document embeddings.

    Returns:
        A ChromaDB vector store containing the embedded listings.
    """
    # Initialize the text splitter
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=500,
        chunk_overlap=20
    )

    # Split the listings into chunks
    chunks = []
    for listing in listings_text:
        chunks.extend(splitter.split_text(listing))

    # Create the ChromaDB vector store
    return Chroma.from_texts(
        texts=chunks,
        embedding=embedding_model,
        collection_name="real_estate_listings"
    )

In [ ]:
# Create a vector store
embedding_model = OpenAIEmbeddings()
vector_store = create_vector_store(listings, embedding_model)

# Create a retriever from vector_store
retriever = vector_store.as_retriever(
    search_type='similarity',
search_kwargs={'k':4}
)

## Chatbot

In [10]:
def load_prompt(path: str = 'chatbot_prompt.txt') -> str:
    """Loads the prompt from a text file.

    Args:
        path (str): The path to the text file containing the prompt.

    Returns:
        str: The content of the prompt file as a single string.

    Raises:
        FileNotFoundError: If the file specified by 'path' is not found.
    """
    try:
        with open(path, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        raise FileNotFoundError(f"Prompt file not found at path: {path}")

In [11]:
def run_chat(chain: ConversationalRetrievalChain) -> None:
    """Runs a real estate chatbot that allows users to query real estate listings.

    The chatbot loads listings from a text file, creates a Chroma vector store,
    creates a ConversationalRetrievalChain, and then allows the user to input
    queries about real estate.

    Args:
        chain: The ConversationalRetrievalChain to use for the chatbot.
    """
    print("=" * 50)
    print("🏡 Real Estate Assistant Chatbot")
    print("Type your preferences to find listings.")
    print("Example: 'I'm looking for a place with 3 bedrooms and a backyard.'")
    print("Type 'exit' or 'quit' to end the conversation.")
    print("=" * 50)

    try:
        while True:
            query = input("You: ")
            if query.lower() in ["exit", "quit"]:
                print("Exiting the assistant. Goodbye!")
                break

            response = chain.invoke({"question": query})
            print(f"\nAssistant: {response['answer']}\n")

    except Exception as e:
        print(f"An error occurred: {e}")

## Run chatbot

In [12]:
def main():
    model_name = 'gpt-4o-mini'
    temperature = 0.5
    amount = 5
    listings_path = 'listings.txt'
    prompt_path = 'chatbot_prompt.txt'

    # Load prompt and setup chain
    prompt = load_prompt(prompt_path)
    prompt_template = PromptTemplate.from_template(prompt)
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key="answer")
    retrieval_llm = ChatOpenAI(model=model_name, temperature=0.0, max_tokens=400)

    chain = ConversationalRetrievalChain.from_llm(
        llm=retrieval_llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={'prompt': prompt_template},
        return_source_documents=True
    )

    # Run the chatbot
    run_chat(chain)


if __name__ == "__main__":
    main()


<ipython-input-12-ce2215a90c9c>:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key="answer")


🏡 Real Estate Assistant Chatbot
Type your preferences to find listings.
Example: 'I'm looking for a place with 3 bedrooms and a backyard.'
Type 'exit' or 'quit' to end the conversation.
You: I'm looking for a place with 3 bedrooms and a backyard.

Assistant: Based on your preference for a place with 3 bedrooms and a backyard, here are two suitable options:

1. **Charming Bungalow in Historic District**  
   - **Location:** Charleston, Ansonborough  
   - **Price:** $525,000  
   - **Bedrooms:** 3  
   - **Bathrooms:** 2  
   - **Square Footage:** 1,500 sq ft  
   - **Key Features:** Original hardwood floors, large backyard, updated kitchen, walking distance to downtown.

2. **Spacious Townhome in Gated Community**  
   - **Location:** Orlando, Lake Nona  
   - **Price:** $400,000  
   - **Bedrooms:** 3  
   - **Bathrooms:** 2.5  
   - **Square Footage:** 1,800 sq ft  
   - **Key Features:** Community clubhouse, pool, modern kitchen, attached garage. (Note: While this townhome has a bac